In [6]:
import polars as pl
import pandas as pd
import numpy as np
import pyreadstat

The path of the network files is in G:/Bevolking/ at CBS

```
path_family = "FAMILIENETWERKTAB"
path_colleagues = "COLLEGANETWERKTAB"
path_neigh = "BURENNETWERKTAB"
path_school = "KLASGENOTENNETWERKTAB"
path_household = "HUISGENOTENNETWERKTAB"
```

These files have three columns:
- "RINPERSOON" the ID of the source
- "RINPERSOONRELATIE" the ID of the target
- "RELATIE", indicating the type of relationship (e.g. in family you can be a brother, or a stepaunt). 

I will be using polars instead of pandas for performance. These are big CSV files, and polars is super fast.


# 1. How to create a network variable
### Simpler case: calculate the share of family members with kids, and the total number of kids

In the same way you can create other "network variables", such as the income of your family members. You'd just need to read the income table and merge it in the last step to calculate the average income.

In [ ]:
# Read the IDs in our sample (to filter the network)
ids = (pl.scan_csv("H:/DATASETS/ids_final.csv") #scan_csv doesn't do anything, just prepare the data to be read
        .select(["RINPERSOON"]) #select column
        .collect() #tell polars to actually read the data
      )
ids.head()

In [ ]:
# Read the network. Keep close family (check the codebook!)
path_family = "FAMILIENETWERKTAB"
year = 2019
#note the curly brackets, they denote a set. 
#Finding an element in a set is much much faster than finding one in a list
close_family = {301, 302, 303, 304, 305, 306, 307, 308, 312, 313, 314, 315, 316, 317, 318, 319} #(step)partners, (step)parents, grandparents, (step)siblings

df_family = (
    pl.scan_csv(f"G:/Bevolking/{path_family}/{path_family[:-3]}{year}TABV1.csv", separator=";")
    .filter(pl.col("RELATIE").is_in(close_family)) #keep only some rows
    .select(["RINPERSOON", "RINPERSOONRELATIE"]) #select columns
    .collect() #actually read data
)

df_family.head()

In [ ]:
# Read the number of kids per person
# We are going to merge the newtork by the family member, 
#so we change the columns name to match it
df_number_kids = pl.read_csv("H:/DATASETS/number_kids_all_people.csv") #read data directly, we want to keep everything
df_number_kids = df_number_kids.rename({"RINPERSOON": "RINPERSOONRELATIE"}) #rename column
df_number_kids.head()

In [ ]:
# Left join to filter out people (egos, rinpersson) in df_family who was not in our initial sample.
# the "alters" (rinpersoonrelatie) can be outside of our sample, e.g. a 50 year old man with children
df_family_filtered = ids.join(df_family, on="RINPERSOON", how="left")
# add the children of your family members
del df_family #free up memory
df_family_filtered = df_family_filtered.join(df_number_kids, on="RINPERSOONRELATIE", how="left")
df_family_filtered.head()

In [ ]:
# Add 3 new binary columns, with the people that had a kid, had a kid in the last 3 and 5 years
# MAKE YOUR OWN COLUMNS, I don't know anything about fertility
df_family_filtered = (df_family_filtered
 .with_columns(pl.col("number_kids").fill_null(0)) #fill missing values
 .with_columns((pl.col("year_last_kid")>2016).alias("kid_last_3").fill_null(0)) #create new column
 .with_columns((pl.col("year_last_kid")>2014).alias("kid_last_5").fill_null(0))
 .with_columns((pl.col("number_kids")>0).alias("at_least_1_kid").fill_null(0))
                     )
df_family_filtered.head()

In [107]:
# Group by person to get the total number of kids, 
#the share of family with a kid in the last 3 and 5 years, 
#and the share of family with at least 1 kid
df_family_filtered = df_family_filtered.groupby("RINPERSOON").agg([
                                            pl.col("number_kids").sum(),
                                            pl.col("kid_last_3").mean(),
                                            pl.col("kid_last_5").mean(),
                                            pl.col("at_least_1_kid").mean()
                                            ])



In [ ]:
# Convert to pandas (slightly inefficient since CBS forgot to install a library. Usually you can just do df_family_filtered.to_pandas()))
# You can merge this with your other features
# You don't need to convert it if you want to save it to disk (df_collega_filtered.to_csv())
df_family_filtered = pd.DataFrame(df_family_filtered.to_dict())
df_family_filtered

# 2. How to create a variable joining two network files
### A bit more complicated case: calculate the share of your colleagues's siblings with kids

This takes ~15 minutes to run. After it's finished please clean up removing unnecessary dataframes, or stopping the kernel if you are done.

If you add other family members it'll increase the computation time.

Other cases are similar: e.g. the families of your families, the families of your colleagues, the colleagues of your family, etc



In [ ]:
# Read the IDs in our population (to filter the network)
ids = (pl.scan_csv("H:/DATASETS/ids_final.csv") #scan_csv doesn't do anything, just prepare the data to be read
        .select(["RINPERSOON"]) #select column
        .collect() #tell polars to actually read the data
      )
ids.head()

In [ ]:
# Read the newtork. Keep siblings only (check the codebook!) and colleagues

path_family = "FAMILIENETWERKTAB"
path_colleagues = "COLLEGANETWERKTAB"
year = 2019

df_family = (
    pl.scan_csv(f"G:/Bevolking/{path_family}/{path_family[:-3]}{year}TABV1.csv", separator=";")
    .filter(pl.col("RELATIE")==306) #keep only some rows (full siblings = 306)
    .select(["RINPERSOON", "RINPERSOONRELATIE"]) #select columns
    .collect() #actually read data
)

df_collega = (
    pl.scan_csv(f"G:/Bevolking/{path_colleagues}/{path_colleagues[:-3]}{year}TABV1.csv", separator=";")
    .select(["RINPERSOON", "RINPERSOONRELATIE"])
    .collect()
)

df_collega.head()

In [8]:
#Huge file! 700M rows
df_collega.shape

(685722380, 2)

In [ ]:
# We are interested in the family of our colleagues. 
#So we will merge df_collega to df_family using RINPERSOONRELATIE of df_collega and RINPERSOON of df_family
df_family = df_family.rename({"RINPERSOON": "RINPERSOONRELATIE", "RINPERSOONRELATIE": "RIN_family_colleague"})
df_family.head(1)

In [ ]:
# Read the number of kids per person
# We are going to merge the newtork by the family member, 
#so we change the columns name to match it
df_number_kids = pl.read_csv("H:/DATASETS/number_kids_all_people.csv") #read data directly, we want to keep everything
df_number_kids = df_number_kids.rename({"RINPERSOON": "RIN_family_colleague"}) #rename column
df_number_kids.head()

In [ ]:
# Left join to filter out people (egos, rinpersson) in df_collega who was not in our initial sample.
df_collega_filtered = ids.join(df_collega, on="RINPERSOON", how="left")
del df_collega #free up memory
# Add the siblings of your colleagues
df_collega_filtered = df_collega_filtered.join(df_family, on="RINPERSOONRELATIE", how="left")
del df_family #free up memory
# add the children of the siblings of your colleagues
df_collega_filtered = df_collega_filtered.join(df_number_kids, on="RIN_family_colleague", how="left")
df_collega_filtered.head()

In [12]:
# Huge dataset!
df_collega_filtered.shape

(624410373, 6)

In [ ]:
# Add 3 new binary columns, with the people that had a kid, had a kid in the last 3 and 5 years
# MAKE YOUR OWN COLUMNS
df_collega_filtered = (df_collega_filtered
 .with_columns(pl.col("number_kids").fill_null(0)) #fill missing values
 .with_columns((pl.col("year_last_kid")>2016).alias("kid_last_3").fill_null(0)) #create new column
 .with_columns((pl.col("year_last_kid")>2014).alias("kid_last_5").fill_null(0))
 .with_columns((pl.col("number_kids")>0).alias("at_least_1_kid").fill_null(0)))
df_collega_filtered.head()

In [ ]:
# Group by person to get the total number of kids, 
#the share of family with a kid in the last 3 and 5 years, 
#and the share of family with at least 1 kid
df_collega_filtered = df_collega_filtered.groupby("RINPERSOON").agg([
                                            pl.col("number_kids").sum(),
                                            pl.col("kid_last_3").mean(),
                                            pl.col("kid_last_5").mean(),
                                            pl.col("at_least_1_kid").mean()
                                            ])

# Convert to pandas (slightly inefficient since CBS forgot to install a library)
# You can merge this with your other features
# You don't need to convert it if you want to save it to disk (df_collega_filtered.to_csv())
df_collega_filtered = pd.DataFrame(df_collega_filtered.to_dict())
df_collega_filtered

# 3. Create school networks
You are only in the school network when you're in school. I'll concatenate your classmates in secondary education

In [7]:
path_school = "KLASGENOTENNETWERKTAB"
# I believe the information of some layers is missing for some years. 
# 504, 505 and 506 (higher education) connect many people (e.g. if you are in economics you'll be connected to every person in your year)
# I'm keeping only secondary education
#{501, 502, 503, 504, 505, 506} #primary, special onderwijs, secondary, prof, prof2, uni 


In [8]:
# Read the IDs in our population (to filter the network)
ids = (pl.scan_csv("H:/DATASETS/ids_final.csv") #scan_csv doesn't do anything, just prepare the data to be read
        .select(["RINPERSOON"]) #select column
        .collect() #tell polars to actually read the data
      )

ids.shape

(6695926, 1)

In [ ]:
%%time
# I'm curious to see how long it takes (it may take too long)

path_school = "KLASGENOTENNETWERKTAB"

# Concatenating all files, merging them in the spot to reduce size
df_school = pl.concat(
    [
        ids.join((pl.scan_csv(f"G:/Bevolking/{path_school}/{path_school[:-3]}{year}TABV1.csv", separator=";")
            .filter(pl.col("RELATIE")==503) #keep secondary school
            .select(["RINPERSOON", "RINPERSOONRELATIE"])
            ).collect(), on="RINPERSOON", how="left")
        for year in range(2009,2020) #concatenate the data for all years
    ]
)

# Remove duplicates
df_school = df_school.unique(subset=["RINPERSOON", "RINPERSOONRELATIE"]) 

# This dataset you can either use it as in the first example 
# (to calculate the kids of your past classmates), or the second example
# (to calculate the kids of the xx of your past classmates)
df_school

In [10]:
# This deletes all variables. I'm doing it to free up the memory for other users.
# Make sure you saved the dataframes you want to save before runnign this!
%reset -f